In [45]:
import pandas as pd
import io
from minio import Minio
import os

METADATA_KEY = "x-amz-meta-type"
METADATA_VAL = "excel_almendros_cercanos"


ACCESS_ROOT = os.environ.get("PREFECT_MINIO_ACCESS_ROOT")
SECRET_ROOT = os.environ.get("PREFECT_MINIO_SECRET_ROOT")
MINIO_HOST = os.environ.get("PREFECT_MINIO_HOST")
minio_client = Minio(MINIO_HOST, access_key=ACCESS_ROOT,
                     secret_key=SECRET_ROOT, secure=False)

In [48]:
# Fetch objects and filter by metadata
objects = minio_client.list_objects(
    "landing-zone", include_user_meta=True)
# print objects name and metadata
objects_name = []
for obj in objects:
    stat = minio_client.stat_object("landing-zone", obj.object_name)
    if stat.metadata.get(METADATA_KEY) == METADATA_VAL:
        objects_name.append(obj.object_name)
# Get objects
for object_name in objects_name:
    data = minio_client.get_object("landing-zone", object_name).read()
    dfTreatments = pd.read_excel(io.BytesIO(data), engine="openpyxl",
                        sheet_name="Tratamientos", na_values=[''])
    dfParcels = pd.read_excel(io.BytesIO(data), engine="openpyxl",
                        sheet_name="Parcelas", na_values=[''])
    # Remove spaces from columns
    dfParcels.columns = dfParcels.columns.astype(str).str.replace(' ', '').str.strip()
    # Convert columns Recinto and Agregado to int
    dfParcels["Recinto"] = pd.to_numeric(dfParcels["Recinto"], downcast='integer', errors='coerce')
    # Remove rows with NaN in Recinto
    dfParcels = dfParcels[dfParcels["Recinto"].notna()]
    # Convert columns Recinto to int
    dfParcels["Recinto"] = dfParcels["Recinto"].astype(int)
    print(dfParcels.dtypes)
    

ProductorNIF          object
Cosecha                int64
ProvinciaId            int64
MunicipioId            int64
Poligono               int64
Parcela                int64
Recinto                int64
Paraje                object
Agregado              object
Zona                   int64
OrdenPAC               int64
SubOrdenPac            int64
SuperficieSIGPAC     float64
SuperficieCultivo    float64
CultivoId              int64
ParcelaVariedadId      int64
SistemaDeRiego        object
RegimenTenenciaId      int64
AñoPlantacion          int64
NºArboles              int64
Marcoplantacionh       int64
Marcoplantacionv       int64
Densidaddesiembra      int64
ATRIA/ADV/ASV         object
ZonaVulnerable        object
ZonaEspecifica        object
UsoParcela            object
Asesoramiento         object
Pendiente%           float64
UHC                    int64
DescripciónUHC        object
ZonaZepa              object
ZonaSIE               object
dtype: object
ProductorNIF          object
